## Extract data from FAME dashboard

### 1) Import required libraries

In [9]:
import requests,json
import pandas as pd
from collections import ChainMap

### 2) Functions to extract data from website

In [73]:
def get_req(url,headers,payload,verification=True): # for old websites with expired ssl certificate, use verification=False (not recommended)
    """
    Get data from the website
    url: website url
    headers: HTTP headers
    payload: Request payload
    verification: ssl verification check 
    
    Returns list of request reponses
    
    """
    reqs=[]
    for i in range(1,39):
        payload_dict={payload:str(i)}
        r=requests.post(url, data=json.dumps(payload_dict), headers=headers,verify=verification)
        reqs.append(r)
    return reqs


In [34]:
def get_json_tables(reqs):
    
    """
    Process the data in json format
    
    reqs: list of request response from pevious step
    
    Returns a list of data in json format
    
    """
    tables=[]
    for request in json_reqs:
        i=json.loads(request.text)
        j=json.loads(i['d'])
        if len(j['Table']) and len(j['Table1'])!=0: 
            dct={}
    
            dct['State_Name']=j['Table1'][0]['State_Name']
            j['Table'].append(dct)
    
            tables.append(j['Table'])
    return tables
    

In [59]:
def process_table(json_tables):
    
    """
    Process the json data
    
    json_table: list of data in json format 
    
    Returns processed json list
    
    """
    
    
    newdict=[]
    newtable=[]
    for v in json_tables:
        for i in range(len(v)):
            if 'Cat_Name' in v[i].keys():
                di={v[i]['Cat_Name']:v[i]['Total']}
            if 'State_Name' in v[i].keys():
                di['State_Name']=v[i]['State_Name']
        
            newdict.append(di)
            cd=dict(ChainMap(*newdict))
        
        
        newtable.append(cd)
        newdict=[]
        
    return newtable
    
    

In [70]:
def write_to_csv(table_list,filename='FAME2_EVsales.csv'): #set csv file name using filename argument
    
    """
    Creates dataframe from json tables and writes to csv file
    
    table_list: processed json tables
    file_name: name of the csv file to be created
    Returns dataframe and writes to csv file
    
    """
    df=pd.DataFrame(table_list)
    col = df.pop("State_Name")
    df.insert(0, col.name, col)
    df=df.fillna(0)
    df['Total']=df.sum(axis=1)
    df=df.rename(columns={'State_Name':'State'})
    df.to_csv(filename)
    
    return df
    

### 3) Run the functions

In [31]:

url = 'https://fame2.heavyindustry.gov.in/Dashboard.aspx/GetStateBywisevichleDetials' #url for FAME 2 website
headers = {'Content-Type': 'application/json'} 
payload='Stateid'
reqs=get_req(url,headers,payload)


In [35]:
tables=get_json_tables(reqs)

In [60]:
new_tables=process_table(tables)

In [71]:
df=write_to_csv(new_tables, index=False)

In [72]:
df #dataframe returned and written to csv file

,State,M1,L1,e-rickshaw,L5M,L2,e-cart,Total
0,Andaman and Nicobar Islands,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Andhra Pradesh,1.0,403.0,7.0,64.0,2.0,0.0,477.0
2,Assam,0.0,13.0,128.0,4.0,0.0,0.0,145.0
3,Bihar,0.0,192.0,671.0,59.0,0.0,0.0,922.0
4,Chandigarh,0.0,20.0,13.0,0.0,0.0,0.0,33.0
5,Chhattisgarh,0.0,249.0,235.0,125.0,0.0,0.0,609.0
6,Daman and Diu,0.0,3.0,0.0,15.0,0.0,0.0,18.0
7,Delhi,503.0,948.0,684.0,0.0,0.0,3.0,2138.0
8,Goa,2.0,13.0,0.0,0.0,0.0,0.0,15.0
9,Gujarat,1.0,270.0,32.0,15.0,0.0,0.0,318.0
